In [1]:
from ase.io import read, write



In [5]:
mols=read('/scratch/phys/sin/Nian_Wu/active_both_au_nacl_latest_version/active_MLIP_nacl_au/step3_generate_aims_input_iter0/selected_configs_for_DFT_final.xyz',index=':')

mols_distorted=read('/scratch/phys/sin/Nian_Wu/active_both_au_nacl_latest_version/active_MLIP_nacl_au/step3_generate_aims_input_iter0/distorted_configs/selected_configs_for_DFT_final_distorted.xyz',index=':')

In [18]:
all_atom_positions = mol.get_positions()


In [ ]:
from scipy.spatial.distance import pdist, cdist
all_atom_positions = mol.get_positions()
dist=cdist(all_atom_positions, all_atom_positions)
np.fill_diagonal(dist, np.inf)

# Find minimal distance for each atom
min_dist = np.min(dist, axis=1).min()

print(min_dist)

0.43185577551560383


In [27]:
min_dists.min()

0.43185577551560383

In [26]:
np.fill_diagonal(dist, np.inf)

# Find minimal distance for each atom
min_dists = np.min(dist, axis=1)

print(min_dists)

[2.81967865 2.81967865 2.81924456 2.81924456 2.8195016  2.8195016
 2.81817129 2.81817129 2.81979036 2.81979036 2.8152136  2.81490797
 2.81490797 2.81449106 2.81449106 2.81548071 2.81548071 2.81648396
 2.81648396 2.8152136  2.8204945  2.8204945  2.81886477 2.81886477
 2.82071143 2.82026463 2.81947183 2.81947183 2.81985306 2.81985306
 2.81838028 2.81882613 2.81882613 2.81908499 2.81908499 2.81913871
 2.81913871 2.81932342 2.81932342 2.81838028 2.81935179 2.81992128
 2.81992128 2.81924648 2.81924648 2.81828395 2.81828395 2.82097739
 2.82056317 2.81935179 2.81607939 2.81607939 2.81539782 2.81539782
 2.81587763 2.81587763 2.82400215 2.82452176 2.81656885 2.81656885
 2.81911596 2.81921688 2.81921688 2.81867996 2.81867996 2.81843683
 2.81843683 2.82197061 2.82197888 2.81911596 2.81529542 2.81529542
 2.81511046 2.81511046 2.81513593 2.81513593 2.82516499 2.82596566
 2.81492825 2.81492825 2.81904373 2.81929432 2.81929432 2.81932917
 2.81932917 2.81864616 2.81864616 2.82253093 2.82241327 2.81904

In [1]:
def check_too_close(mols, threshold=0.5):
    for i, mol in enumerate(mols):
        positions = mol.get_positions()
        num_atoms = len(mol)
        too_close = False
        for j in range(num_atoms):
            for k in range(j + 1, num_atoms):
                distance = ((positions[j] - positions[k]) ** 2).sum() ** 0.5
                if distance < threshold:
                    print(f"Configuration {i} has atoms {j} and {k} too close: {distance:.2f} Å")
                    too_close = True
        if not too_close:
            print(f"Configuration {i} has no atoms too close.")


In [9]:
mol=mols[26]

In [12]:
distort = Distort(mol)
method = np.random.choice(3, size=1, replace=False)[0]

mol_distorted = distort.apply_bond_distort(sd=0.1)



Distorting bond 2.66638838430609 between atoms 320 (Z=17) and 322 (Z=11)
Distorting bond 2.8641255446651783 between atoms 322 (Z=11) and 320 (Z=17)
Distorting bond 2.6556732593743146 between atoms 324 (Z=17) and 326 (Z=11)
Distorting bond 2.0532794046383787 between atoms 326 (Z=11) and 324 (Z=17)
Distorting bond 2.645474331332042 between atoms 328 (Z=17) and 330 (Z=11)
Distorting bond 2.422533846051896 between atoms 330 (Z=11) and 328 (Z=17)
Distorting bond 2.646655856492116 between atoms 332 (Z=17) and 334 (Z=11)
Distorting bond 2.981753858698244 between atoms 334 (Z=11) and 332 (Z=17)
Distorting bond 2.645408956459861 between atoms 336 (Z=17) and 338 (Z=11)
Distorting bond 3.318718649995454 between atoms 338 (Z=11) and 336 (Z=17)
Distorting bond 2.6426212834511538 between atoms 340 (Z=17) and 342 (Z=11)
Distorting bond 2.770606644082489 between atoms 342 (Z=11) and 340 (Z=17)
Distorting bond 2.651997576190877 between atoms 344 (Z=17) and 346 (Z=11)
Distorting bond 2.603841921914714 b

In [16]:
write('mol_26.in', mol_distorted, format='aims')

In [17]:
write('selected_configs_for_DFT_final_distorted.xyz', mols_distorted, format='extxyz')

In [15]:
mols_distorted[26] = mol_distorted

In [13]:
check_too_close([mol])

Configuration 0 has atoms 638 and 651 too close: 0.43 Å


In [10]:
import ase.neighborlist
import numpy as np
from itertools import combinations

from ase.neighborlist import neighbor_list, natural_cutoffs, NeighborList
import argparse
import os





np.random.seed(0)

class Distort:
    def __init__(self, atoms):
        self.atoms = atoms

    def apply_rattle(self, sd = 0.1):
        # Apply random distortion to atomic positions
        atoms_rattle = self.atoms.copy()
        rattle = np.random.normal(0, sd, atoms_rattle.positions.shape)
        atoms_rattle.positions += rattle

        return atoms_rattle
    def apply_bond_distort(self, sd=0.1, threshold=0.7):
        # Apply random distortion to bond lengths
        atoms_bond_distort = self.atoms.copy()
        
        # Define cutoffs automatically (element-dependent radii)
        cutoffs = natural_cutoffs(atoms_bond_distort)

# Build neighbor list
        i, j = neighbor_list("ij", atoms_bond_distort, cutoffs)

        for idx1, idx2 in zip(i, j):

            atomic_number_idx1 = atoms_bond_distort.get_atomic_numbers()[idx1]
            atomic_number_idx2 = atoms_bond_distort.get_atomic_numbers()[idx2]
            

            distance = atoms_bond_distort.get_distance(idx1, idx2)
            
            print(f"Distorting bond {distance} between atoms {idx1} (Z={atomic_number_idx1}) and {idx2} (Z={atomic_number_idx2})")
            distortion = np.random.normal(0, sd) * distance
            new_distance = distance + distortion
            count = 0
            while new_distance < threshold and count<10:  # Prevent unphysical bond lengths
                print(f"Warning: Bond length {new_distance:.3f} Å is below threshold {threshold:.3f} Å. Redistorting...")
                distortion = np.random.normal(0, sd) * distance
                new_distance = distance + distortion
                count += 1
                
            if count ==10:
                if distance < threshold:
                    new_distance=threshold  # Ensure bond length is above threshold
                else:
                    new_distance=distance  # Reset to original distance if too many attempts
                print(f"Resetting bond length to original value {distance:.3f} Å after 10 attempts.")

            atoms_bond_distort.set_distance(idx1, idx2, new_distance, fix=0.5)

        return atoms_bond_distort

    def apply_angle_distort(self, sd=0.001):
        """Randomly distort bond angles"""
        atoms_angle_distort = self.atoms.copy()

        cutoffs = natural_cutoffs(atoms_angle_distort)
        nl = NeighborList(cutoffs, self_interaction=False, bothways=True)
        nl.update(atoms_angle_distort)

        for j in range(len(atoms_angle_distort)):
            neighbors, _ = nl.get_neighbors(j)
            for i, k in combinations(neighbors, 2):
                angle = atoms_angle_distort.get_angle(i, j, k)
                distortion = np.random.normal(0, sd) * angle
                new_angle = angle + distortion
                # Build boolean mask
                mask = [False] * len(atoms_angle_distort)
                mask[k] = True
                
                if np.abs(new_angle) < 1 or np.abs(new_angle - 180) < 1:
                    pass

                else:
                    atoms_angle_distort.set_angle(i, j, k, new_angle, mask=mask)

        return atoms_angle_distort


    def apply_torsion_distort(self, sd=5.0):
        """Randomly distort dihedral (torsion) angles"""
        atoms_torsion_distort = self.atoms.copy()

        cutoffs = natural_cutoffs(atoms_torsion_distort)
        nl = NeighborList(cutoffs, self_interaction=False, bothways=True)
        nl.update(atoms_torsion_distort)

        # Loop over bonds j-k as central axis
        for j in range(len(atoms_torsion_distort)):
            neighbors_j, _ = nl.get_neighbors(j)
            for k in neighbors_j:
                neighbors_k, _ = nl.get_neighbors(k)

                # i is neighbor of j (excluding k), l is neighbor of k (excluding j)
                for i in neighbors_j:
                    if i == k:
                        continue
                    for l in neighbors_k:
                        if l == j:
                            continue
                        # Valid torsion i-j-k-l
                        dihedral = atoms_torsion_distort.get_dihedral(i, j, k, l)
                        distortion = np.random.normal(0, sd)  # in degrees
                        new_dihedral = dihedral + distortion
                        # Move atom l and its "side"
                        mask = [False] * len(atoms_torsion_distort)
                        mask[l] = True
                        atoms_torsion_distort.set_dihedral(i, j, k, l, new_dihedral, mask=mask)

        return atoms_torsion_distort


In [8]:
check_too_close(mols)

Configuration 0 has no atoms too close.
Configuration 1 has no atoms too close.
Configuration 2 has atoms 638 and 650 too close: 0.40 Å
Configuration 2 has atoms 639 and 646 too close: 0.37 Å
Configuration 2 has atoms 644 and 656 too close: 0.37 Å
Configuration 2 has atoms 645 and 660 too close: 0.40 Å
Configuration 3 has atoms 638 and 646 too close: 0.37 Å
Configuration 3 has atoms 639 and 650 too close: 0.39 Å
Configuration 3 has atoms 644 and 660 too close: 0.39 Å
Configuration 3 has atoms 645 and 656 too close: 0.37 Å
Configuration 4 has no atoms too close.
Configuration 5 has no atoms too close.
Configuration 6 has no atoms too close.
Configuration 7 has no atoms too close.
Configuration 8 has no atoms too close.
Configuration 9 has no atoms too close.
Configuration 10 has no atoms too close.
Configuration 11 has no atoms too close.
Configuration 12 has no atoms too close.
Configuration 13 has no atoms too close.
Configuration 14 has no atoms too close.
Configuration 15 has no ato

In [7]:
check_too_close(mols_distorted)

Configuration 0 has no atoms too close.
Configuration 1 has no atoms too close.
Configuration 2 has no atoms too close.
Configuration 3 has atoms 638 and 646 too close: 0.37 Å
Configuration 3 has atoms 639 and 650 too close: 0.39 Å
Configuration 3 has atoms 644 and 660 too close: 0.39 Å
Configuration 3 has atoms 645 and 656 too close: 0.38 Å
Configuration 4 has no atoms too close.
Configuration 5 has no atoms too close.
Configuration 6 has no atoms too close.
Configuration 7 has no atoms too close.
Configuration 8 has no atoms too close.
Configuration 9 has no atoms too close.
Configuration 10 has no atoms too close.
Configuration 11 has no atoms too close.
Configuration 12 has no atoms too close.
Configuration 13 has no atoms too close.
Configuration 14 has no atoms too close.
Configuration 15 has no atoms too close.
Configuration 16 has no atoms too close.
Configuration 17 has no atoms too close.
Configuration 18 has no atoms too close.
Configuration 19 has no atoms too close.
Config